In [1]:
import pylast
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pymongo
from pymongo import MongoClient
import logging
import auth

In [2]:
# Create logger
logger = logging.getLogger('lastfm_logger')
logger.setLevel(logging.DEBUG)

# Create handlers
f_handler = logging.FileHandler('debug.log')
f_handler.setLevel(logging.DEBUG)

# Logging formatter
f_format = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
f_handler.setFormatter(f_format)

logger.addHandler(f_handler)

In [3]:
API_KEY = auth.API_KEY
API_SECRET = auth.API_SECRET
username = auth.username
password = auth.password
password_hash = auth.password_hash

In [4]:
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET, username=username, password_hash=password_hash)
logger.info('Network connection established')

In [18]:
mongo = MongoClient()

db = mongo['project-05']

db_user = db['Usernames']

In [19]:
print(db.list_collection_names())

[]


In [20]:
url_base = 'http://last.fm/music/artist/+listeners?page='
artist_list = ['Taylor Swift', 'Selena Gomez', 'Lorde', 'Halsey', 'Lana Del Ray', 'Marina & The Diamonds', 'Tove Lo', 
               'Charli XCX', 'Lady Gaga', 'Gwen Stefani']
url_list = []

for artist in artist_list:
    for i in range(1, 10):
        url_list.append(url_base.replace('artist',artist.replace(' ', '+'))+str(i))

In [21]:
def follower_scrape(url):
    username_list = []
    
    browser.get(url)
    
    for i in range(1,32):
        xpath = '//*[@id="mantle_skin"]/div[3]/div[2]/div[1]/div/ol/li[{}]/div/h3/a'.format(i)
        try:
            temp_username = browser.find_elements_by_xpath(xpath)[0].text
            username_list.append(temp_username)
            logger.debug(f'{i} completed')
        except IndexError:
            logger.error(f'Index error at {i}, skipping...')
    
    return username_list

In [22]:
# test = follower_scrape(url_list[0], username, password)
result_list = []

browser = webdriver.Firefox()
browser.get('http://secure.last.fm/login')

username1 = browser.find_element_by_id('id_username')
password1 = browser.find_element_by_id('id_password')
submit = browser.find_element_by_name('submit')

username1.send_keys(username)
password1.send_keys(password)

submit.click()

for url in url_list:    
    result_list = result_list + follower_scrape(url)
    
browser.quit()

In [23]:
un_list = set(result_list)

In [24]:
def user_top_tracks(username, limit, playcount):
    try:
        user = network.get_user(username)

        top_temp = user.get_top_tracks(limit=limit)
        top = []

        if limit > len(top_temp):
            limit = len(top_temp)

        for i in range(limit):
            if top_temp[i].weight <= playcount:
                logger.debug('Song Range Limit reached; terminating process...')
                break
            else:
                top.append((top_temp[i].item.title, top_temp[i].item.artist.get_name(), top_temp[i].weight))
                logger.debug(f'{i + 1} completed, {limit - i - 1} remaining')

        return top
    except:
        logger.error('User not found')
        pass

In [25]:
for follower in un_list:
    db_user.insert_one({'Username': follower, 'Tracks': user_top_tracks(follower, 100, 10)})